# Исследование надежности заемщиков



## Цель исследования

**Заказчик** — кредитный отдел банка. 

Нужно проверить 4 гипотезы:
1. Семейное положение влияет на факт погашения кредита в срок
2. Количество детей клиента влияет на факт погашения кредита в срок
3. Уровень дохода влияет на возврат кредита в срок
4. Разные цели кредита влияют на возврат кредита в срок

Входные данные от банка — статистика о платёжеспособности клиентов.
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Ход исследования

1. **Обзор данных**


2. **Предобработка даных**

 
   2.1 Заполнение пропусков
 
   2.2 Проверка данных на аномалии и исправления
 
   2.3 Изменение типов данных
 
   2.4 Удаление дубликатов
 
   2.5 Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма
 
   2.6 Категоризация дохода
 
   2.7 Категоризация целей кредита


3. **Проверка гипотез**



### Шаг 1. Обзор данных

In [1]:
import pandas as pd
try:
    df_clients = pd.read_csv('/datasets/data.csv')
except:
     df_clients = pd.read_csv('data.csv')

df_clients.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


На этом этапе видим следующее:
* В столбцах days_employed и total_income	есть пропущенные значения

* В столбцах days_employed и total_income указаны вещественные значения, для упрощения расчетов необходимо заменить их на целочисленные

* В столбце education не везде соблюден нижний регистр

* В days_employed есть отрицательные значения (это может быть - если написали стаж через дефис). Возможно данные станут пригодны для работы, если убрать у них отрицательное значение.


In [2]:
df_clients.info() #получим общую информацию о таблице:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [3]:
df_clients.isna().sum() #убедимся, что данные пропущены только в столбцах days_employed и total_income и посмотрим сколько их всего

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Действительно, данные пропущены только в двух столбцах: days_employed и total_income. 

Данные пропущены в 10% строк в этих столбцах, что является слишком значительным, чтобы просто удалить пропуски.
Получим, дополнительную информацию о таблице:

In [4]:
df_clients.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


Дополнительно обнаружили:

* children - количество детей от -1 до 20, считаем человеческой ошибкой(выглядит, что на самом деле -1 = 1, а 20=2)

* dob_years - возраст равен 0 - некорректные данные при заполнении информации, нужно будет разбираться, по какой причине

* days_employed - максимальный стаж 401755/365 - 1100 лет Оо

### Шаг 2.1 Заполнение пропусков


Разница в доходах даже на уровне одной группы, например предпринимателя - может быть огромной. Потому для заполнения пропусков лучше взять медиану.


In [5]:
#нашли медиану по каждой группе
df_clients.groupby('income_type')['total_income'].median()

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        150447.935283
компаньон          172357.950966
пенсионер          118514.486412
предприниматель    499163.144947
сотрудник          142594.396847
студент             98201.625314
Name: total_income, dtype: float64

Компаньон зарабатывает значительно меньше предпринимателя, не стоит объединять его с предпринимателем.

Интересная штука- у безработного есть доход.
Безработный и пенсионер - могут быть одной категорией, либо безработный стал безработным недавно,попал под сокращение.



In [6]:
df_clients['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

Количества безработных, предпринимателей, студентов и тех, кто в декрете- недостаточно для того, чтобы сделать верные выводы по данным. Их можно удалить.

In [7]:
# удаляем
df_clients = df_clients[(df_clients["income_type"] != 'безработный')&
                       (df_clients["income_type"] != 'предприниматель')&
                       (df_clients["income_type"] != 'студент')&
                       (df_clients["income_type"] != 'в декрете')]
#проверяем
df_clients['income_type'].value_counts()

сотрудник      11119
компаньон       5085
пенсионер       3856
госслужащий     1459
Name: income_type, dtype: int64

In [8]:
#заполняем медианой по каждой группе
df_clients['total_income']=df_clients.groupby('income_type')['total_income'].apply(lambda x: x.fillna(x.median()))

#убедились, что мы заполнили пропуски в данных в столбце income_type медианными значениями 
df_clients.isna().sum() 

children               0
days_employed       2173
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
dtype: int64

Все ок, с income_type разобрались:)

### Шаг 2.2 Проверка данных на аномалии и исправления.

Пришло время разобраться с отрицательными значениями в трудовом стаже. Заменим их на положительные и посчитаем средние значения по каждому типу занятости.

In [9]:
df_clients['days_employed'] = df_clients['days_employed'].abs()
df_clients['days_employed'].head() #убедились, что избавились от минусов

0      8437.673028
1      4024.803754
2      5623.422610
3      4124.747207
4    340266.072047
Name: days_employed, dtype: float64

In [10]:
#нашли среднее по каждой группе
df_clients.groupby('income_type')['days_employed'].mean() 


income_type
госслужащий      3399.896902
компаньон        2111.524398
пенсионер      365003.491245
сотрудник        2326.499216
Name: days_employed, dtype: float64

Явно выделяется пенсионер со стажем в тысячу лет.

Посмотрим на остальных: у компаньона всего 5 лет, у сотрудника - 6 при среднем возрасте 40 лет(считаем в слудующем коде). 

Данные не корректны по всему столбцу, потому мы не будем заменять пропуски на среднее значение, а сделаем новый столбец, в котором предположим, что люди работали с 18 лет без прерывания трудового стажа.

Сначала разберемся с нулевым возрастом:


In [11]:
len(df_clients[df_clients['dob_years']==0])

101

0,5% без возраста - слишком много, чтобы просто удалить. заменим средним по типу занятости:

In [12]:
years_mean = df_clients.groupby('income_type')['dob_years'].mean() 
years_mean #результат выглядит адекватно

income_type
госслужащий    40.636737
компаньон      39.697542
пенсионер      59.063019
сотрудник      39.821027
Name: dob_years, dtype: float64

In [13]:
#заменим 0 на среднее значение по группам
df_clients['dob_years'] = df_clients['dob_years'].fillna(value=0) 
df_clients.loc[df_clients['dob_years']==0, 'dob_years'] = years_mean
df_clients['dob_years'] = df_clients.groupby('income_type')['dob_years'].apply(lambda x: x.fillna(x.mean()))

# проверим, что все ок, нет пропущенных значений по возрасту
df_clients.isna().sum()

children               0
days_employed       2173
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
dtype: int64

In [14]:
# проверим, что все ок, значения подставились
df_clients['dob_years'].sort_values().head(5)

13021    19.0
15659    19.0
12047    19.0
20230    19.0
2725     19.0
Name: dob_years, dtype: float64

Теперь посчитаем среднее значение по стажу, из расчета, что все начали работать с 18 лет и в году 365 дней. Заменим некорретные данные столбца корректными значениями по среднему. 


In [15]:
df_clients['days_employed'] =(df_clients['dob_years']-18)*365
# проверим, что все ок
df_clients.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8760.0,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,6570.0,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5475.0,33.0,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,5110.0,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,12775.0,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [16]:
# Теперь данные похожи на правду. Посчитаем среднее по каждой группе:
df_clients.groupby('income_type')['days_employed'].mean() 

income_type
госслужащий     8323.657949
компаньон       7976.817374
пенсионер      15100.400156
сотрудник       8036.927874
Name: days_employed, dtype: float64

Заполним пропуски средним по каждой группе и убедимся, что пропусков не осталось:

In [17]:
df_clients['days_employed'] = df_clients.groupby('income_type')['days_employed'].apply(lambda x: x.fillna(x.mean()))


In [18]:
df_clients.isna().sum() # теперь пропусков нет ни в  одном столбце

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Заменим некорректное число детей:

In [19]:
df_clients['children'] = df_clients['children'].replace(20, 2)
df_clients['children'] = df_clients['children'].replace(-1, 1)

# проверили, что все ок
df_clients['children'].min()
df_clients['children'].max()

5

Приведем столбец education к нижнему регистру:

In [20]:
df_clients['education'] = df_clients['education'].str.lower()
df_clients['education'].head() #проверили 

0     высшее
1    среднее
2    среднее
3    среднее
4    среднее
Name: education, dtype: object

In [21]:
# Разберемся со столбцом gender
df_clients['gender'].value_counts()

F      14233
M       7285
XNA        1
Name: gender, dtype: int64

Мы видим, что кредиты почти в два раза больше берут женщины, чем мужчины и есть 1 человек с неопределенным полом, но так как данные не будут участвовать в нашей дальнейшей работе, оставим как есть. 

Исправили все аномалии, бросающиеся в глаза и заполнили пропуски корректными данными:
* привели все к нижнему регистру
* разобрались с 0 возрастом
* разобрались с некорретными данными по стажу (отрицательные и слишком большие максимальные значения)
* устранили опечатку про количество детей = -1 и 20
* устранили категории, в которых слишком мало данных и которые только исказят картину

Можем переходить к следующему шагу.


### Шаг 2.3. Изменение типов данных

Для анализа данных нам не нужны вещественные значения, достаточно целочисленных. Это упростит расчеты и визуальное восприятие.

In [22]:
df_clients['days_employed'] = df_clients['days_employed'].astype('int')
df_clients['total_income'] = df_clients['total_income'].astype('int')
df_clients.info() #проверили, что все ок

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21519 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21519 non-null  int64  
 1   days_employed     21519 non-null  int32  
 2   dob_years         21519 non-null  float64
 3   education         21519 non-null  object 
 4   education_id      21519 non-null  int64  
 5   family_status     21519 non-null  object 
 6   family_status_id  21519 non-null  int64  
 7   gender            21519 non-null  object 
 8   income_type       21519 non-null  object 
 9   debt              21519 non-null  int64  
 10  total_income      21519 non-null  int32  
 11  purpose           21519 non-null  object 
dtypes: float64(1), int32(2), int64(4), object(5)
memory usage: 2.0+ MB


### Шаг 2.4. Удаление дубликатов.

На этом шаге важно проверить на явные дубликаты и неявные в столбцах с категориальными значениями.
Мы видим, что у нас есть:
* income_type - по этому столбцу мы уже смотрели данные, просто разобьем на категории в следующем шаге
* у family_status есть family_status_id - там все понятно, или человек в браке или нет, поэтому на дубли смотреть не требуется
* purpose - цели кредита, нужно будет разобраться с дублями

In [23]:
# проверили на явные дубликаты, что случайно ничего не продублировалось
df_clients.duplicated().sum() 

71

In [24]:
#проверили на неявные дубликаты:
purpose = df_clients['purpose'].unique()
purpose.sort()
purpose


array(['автомобили', 'автомобиль', 'высшее образование',
       'дополнительное образование', 'жилье',
       'заняться высшим образованием', 'заняться образованием',
       'на покупку автомобиля', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля', 'на проведение свадьбы',
       'недвижимость', 'образование', 'операции с жильем',
       'операции с коммерческой недвижимостью',
       'операции с недвижимостью', 'операции со своей недвижимостью',
       'покупка жилой недвижимости', 'покупка жилья',
       'покупка жилья для сдачи', 'покупка жилья для семьи',
       'покупка коммерческой недвижимости', 'покупка недвижимости',
       'покупка своего жилья', 'получение высшего образования',
       'получение дополнительного образования', 'получение образования',
       'приобретение автомобиля', 'профильное образование',
       'ремонт жилью', 'свадьба', 'свой автомобиль',
       'сделка с автомобилем', 'сделка с подержанным автомобилем',
       'строительство 

Видим, что цель есть коммерческая и жилая недвидимость, и есть отдельная категория -недвижимость, которая неизвестно куда относится, потому логично все их отнести к недвижимости. Есть категория - ремонт жилья, ее нельзя отнести к ипотеке, и в то же время кредит на свадьбу встречается гораздо чаще. Предварительно можем выделить такие категории:

* автомобиль
* образование
* недвижимость
* личные цели

Посмотрим, сколько встречается целей с ремонтом жилья?

In [25]:
df_clients['purpose'].value_counts()

свадьба                                   797
на проведение свадьбы                     776
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
операции с жильем                         653
покупка жилья для сдачи                   652
операции с коммерческой недвижимостью     651
покупка жилья                             647
жилье                                     647
покупка жилья для семьи                   641
строительство собственной недвижимости    634
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
строительство недвижимости                620
покупка своего жилья                      620
ремонт жилью                              611
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

Ремонт жилья - 611, это слишком мало, чтоб делать отдельную категорию, можно включить ремонт жилья и свадьбу в общую категорию- на личные цели.

In [26]:
# функция для замены неявных дубликатов в категории "автомобиль"
def replace_wrong_values(wrong_values, correct_value): 
    for wrong_value in wrong_values: 
        df_clients['purpose'] = df_clients['purpose'].replace(wrong_value, correct_value) 

duplicates = ['автомобили','на покупку автомобиля','на покупку подержанного автомобиля', 'на покупку своего автомобиля',
             'приобретение автомобиля', 'свой автомобиль', 'сделка с автомобилем','сделка с подержанным автомобилем' ] 
purpose = 'автомобиль'
replace_wrong_values(duplicates, purpose)


# функция для замены неявных дубликатов в категории "образование"
def replace_wrong_values(wrong_values, correct_value): 
    for wrong_value in wrong_values: 
        df_clients['purpose'] = df_clients['purpose'].replace(wrong_value, correct_value) 

duplicates = ['высшее образование', 'дополнительное образование', 'заняться высшим образованием', 'заняться образованием',
              'получение высшего образования', 'получение дополнительного образования', 'получение образования',
             'профильное образование'] 
purpose = 'образование'
replace_wrong_values(duplicates, purpose)


# функция для замены неявных дубликатов в категории "недвижимость"
def replace_wrong_values(wrong_values, correct_value): 
    for wrong_value in wrong_values: 
        df_clients['purpose'] = df_clients['purpose'].replace(wrong_value, correct_value) 

duplicates = ['жилье','операции с жильем','операции с коммерческой недвижимостью',
       'операции с недвижимостью', 'операции со своей недвижимостью',
       'покупка жилой недвижимости', 'покупка жилья',
       'покупка жилья для сдачи', 'покупка жилья для семьи',
       'покупка коммерческой недвижимости', 'покупка недвижимости',
       'покупка своего жилья', 'строительство жилой недвижимости', 'строительство недвижимости',
       'строительство собственной недвижимости'] 
purpose = 'недвижимость'
replace_wrong_values(duplicates, purpose)

# функция для замены неявных дубликатов в категории "личные цели"
def replace_wrong_values(wrong_values, correct_value): 
    for wrong_value in wrong_values: 
        df_clients['purpose'] = df_clients['purpose'].replace(wrong_value, correct_value) 

duplicates = ['на проведение свадьбы', 'ремонт жилью', 'свадьба', 'сыграть свадьбу'] 
purpose = 'личные цели'
replace_wrong_values(duplicates, purpose)

df_clients['purpose'].value_counts() # проверили, что ничего не забыли



недвижимость    10225
автомобиль       4314
образование      4022
личные цели      2958
Name: purpose, dtype: int64

У нас осталось 4 категории, по которым видно, что самый популярный кредит- ипотека.

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

Для того, чтобы ответить на поставленные вопросы, нам нужно выполнить следующие категоризации:
* категоризация по образованию

* категоризация по семейному положению

* категоризация по наличию детей

* категоризация по целям кредита (уже сделали в предыдущем шаге)


In [27]:
# категоризация по образованию
education_category = df_clients[['education','education_id']]
education_category = education_category.drop_duplicates().reset_index(drop=True)

education_category.sort_values('education_id')


,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


In [28]:
# категоризация по семейному положению 
family_category = df_clients[['family_status','family_status_id']]
family_category = family_category.drop_duplicates().reset_index(drop=True)
family_category.sort_values('family_status_id')

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,Не женат / не замужем,4


In [29]:
# категоризация по наличию детей:

def children_category(children):
    try:
        if children ==0:
            return 'нет детей'
        if children == 1:
            return 'есть ребенок'
        if children ==2:
            return 'есть 2 ребенка'
        if children >=3:
            return 'многодетные'
           
        
    except: 'что-то пошлло не так'
        
df_clients['children_category'] = df_clients['children'].apply(children_category)
df_clients['children_category'].value_counts()


нет детей         14145
есть ребенок       4864
есть 2 ребенка     2130
многодетные         380
Name: children_category, dtype: int64

In [30]:
df_clients['debt'].value_counts() # - не имел задолженность по кредиту - это 0, имел - 1

0    19780
1     1739
Name: debt, dtype: int64

### Шаг 2.6. Категоризация дохода.

На основании диапазонов, указанных в задании создаем столбец с категориями
Используем конструкцию try...except для того, чтобы выполнение кода не прервалось.

In [31]:
def total_income_category(total_income):
    try:
        if total_income <=30000:
            return 'E'
        if total_income < 50000:
            return 'D'
        if total_income <= 200000:
            return 'C'
        if total_income <= 1000000:
            return 'B'
        if total_income >1000001:
            return 'A'
    except: 'что-то пошлло не так'
        
df_clients['total_income_category'] = df_clients['total_income'].apply(total_income_category)
df_clients['total_income_category'].value_counts() 


C    16083
B     5039
D      350
A       25
E       22
Name: total_income_category, dtype: int64

### Шаг 2.7. Категоризация целей кредита.

Уже категоризировали на прошлом шаге.
В инструкциях к заданию - рекомендуется оставить категорию - 'проведение свадьбы'.
Не согласна с этим, исходя из данных получается, что:
* есть траты на образование
* ипотека (на строяющееся/готовое жилье и коммерческую недвижимость
* авто
* да, есть много кредитов на свадьбу. но так же есть кредиты на ремонт жилья - 612, что точно нельзя отнести к ипотеке. поэтому оставляю цель кредита - личные цели, а выданный кредит будет потребительским.


### Ответы на вопросы.

##### Вопрос 1: Есть ли зависимость между количеством детей и возвратом кредита в срок?


In [32]:
df_children_pivot = df_clients.pivot_table(index=['debt'],columns='children_category',values= 'family_status_id',aggfunc='count')
df_children_pivot 

children_category,есть 2 ребенка,есть ребенок,многодетные,нет детей
debt,,,,
0,1929,4420,349,13082
1,201,444,31,1063


##### Вывод 1:

В целом те, у кого нет детей берут больше кредитов.

* нет детей - платят вовремя 92%

* 1 ребенок - платят вовремя 90%

* 2 ребенка - платят вовремя 91%

* многодетные - платят вовремя 92%

По моим расчетам выходит, что те, у кого нет детей и многодетные - платят банку вовремя чаще, чем те, у кого 1 или 2 ребенка. Но возможно, выборка в 349 многодетных людей - не показывает реального положения вещей. Точно можем сказать, что те, у кого нет детей платят лучше, чем те у кого они есть.

##### Вопрос 2: Есть ли зависимость между семейным положением и возвратом кредита в срок?


In [33]:
df_family_pivot = df_clients.pivot_table(index=['debt'],columns='family_status',values= 'family_status_id',aggfunc='count')
df_family_pivot

family_status,Не женат / не замужем,в разводе,вдовец / вдова,гражданский брак,женат / замужем
debt,,,,,
0,2538,1110,897,3787,11448
1,274,85,63,388,929


##### Вывод 2:

* 90% Холостых платит кредит вовремя
* 93%  Разведенных платит кредит вовремя
* 94% вдовцов платит кредит вовремя
* 91% тех, кто в гражданской браке платит кредит вовремя
* 93% женатых платит кредит вовремя


Вдовцы почему-то на первом месте по выплатам вовремя (возможно всему виной недостаточное количество вдовцов для анализа).

На втором месте -  женатые и разведенные - вовремя платят 93%

Те, кто в гражданском браке- вовремя платят 91% (менее ответственные, чем разведенные).

Холостые ожидаемо хуже всех - 90% платит вовремя.




##### Вопрос 3: Есть ли зависимость между уровнем дохода и возвратом кредита в срок?


In [34]:
df_income_pivot = df_clients.pivot_table(index=['debt'],columns='total_income_category',values= 'family_status_id', aggfunc='count')
df_income_pivot

total_income_category,A,B,C,D,E
debt,,,,,
0,23,4683,14725,329,20
1,2,356,1358,21,2


##### Вывод 3:

* A 92% платит вовремя

* B 93% платит вовремя

* C 92% платит вовремя

* D 94% платит вовремя

* E 91% платит вовремя

По расчетам выходит, что те, у кого доход от 30 до 50 тыс руб платят лучше, чем те, у когодоход выше.
Возможно, выборки в 329 людей недостаточно, чтобы сформировать объективную картину. ТО же с категорией А и E.

Точно можем сказать, судя по категориям B и C, что чем выше доход, тем больше вероятность, что клиент будет платить вовремя.

##### Вопрос 4: Как разные цели кредита влияют на его возврат в срок?


In [35]:
df_purpose_pivot = df_clients.pivot_table(index=['debt'],columns='purpose',values= 'family_status_id', aggfunc='count')
df_purpose_pivot

purpose,автомобиль,личные цели,недвижимость,образование
debt,,,,
0,3912,2737,9479,3652
1,402,221,746,370


##### Вывод 4:


Ипотека - лидер в причинах взятия кредита.

Автомобиль -91% возвращает кредит в срок
Личные цели - 93% возвращает кредит в срок
Недвижимость - 93% возвращает кредит в срок
Образование - 90% возвращает кредит в срок

Личные цели(чаще расходы на свадьбу) и недвижимость -по таким кредитам люди выплачивают кредит вовремя лучше всего.
На втором месте  - автомобиль, на третьем образование.

## Общий вывод:

Для того, чтобы проверить гипотезы - осуществили проверку данных на аномалии и исправления.
Разобрались с некорректными:
* стажем - заменили средними значениями по группе из расчета, что люди работали с 18 лет
* возрастом - нули заменили на средние значения по группе
* количеством детей - опечатки, где детей было 20 и -1 исправили на 2 и 1

Изменили типы данных на целочисленные, так как для анализа этого достаточно и это упростит визуальное восприятие.
Удалили явные и неявные дубликаты.

Осуществили категоризацию целей кредита на:
* автомобиль
* образование
* недвижимость
* личные цели

Осуществили категоризацию по семейному положению:
* женат / замужем
* гражданский брак	
* вдовец / вдова	
* в разводе	
* Не женат / не замужем	

Осуществили категоризацию по наличию детей:
* нет детей        
* есть ребенок      
* есть 2 ребенка     
* многодетные  

Осуществили категоризацию дохода:

 - 0–30000 — 'E';
 - 30001–50000 — 'D';
 - 50001–200000 — 'C';
 - 200001–1000000 — 'B';
 - 1000001 и выше — 'A'.
 
**Пришли к следующим выводам:**
 
1. Те, у кого нет детей платят лучше, чем те, у кого они есть. 

* нет детей - платят вовремя 92%
* 1 ребенок - платят вовремя 90%
* 2 ребенка - платят вовремя 91%
* многодетных в расчет не берем, из-за недостаточной выборки

2. Те, кто в браке или находился в браке платят лучше, чем холостые.

* женатые и разведенные - вовремя платят 93%
* в гражданском браке- вовремя платят 91% (менее ответственные, чем разведенные).
* холостые ожидаемо хуже всех - 90% платит вовремя
* вдовцов в расчет не берем из-за недостаточной выборки

3. Чем выше доход, тем больше вероятность, что клиент будет платить вовремя


4. Ипотека - лидер и по цели кредита и по его своевременному погашению.


1. Недвижимость и личные цели(чаще расходы на свадьбу) - 93% платят вовремя

2. Автомобиль - 91% платит вовремя.

3. Образование - 90% платит вовремя.

